In [ ]:
import cv2
import os
import numpy as np

# Définir les connexions entre les articulations
skeleton_connections = [
    (0, 1), (1, 2), (2, 6),  # Jambe droite
    (5, 4), (4, 3), (3, 6),  # Jambe gauche
    (6, 7), (7, 8), (8, 9),  # Colonne vertébrale
    (10, 11), (11, 12), (12, 7),  # Bras droit
    (15, 14), (14, 13), (13, 7)   # Bras gauche
]

def draw_skeleton_with_action(image_dir, annotation, release_data, output_dir):
    """
    Dessine les squelettes humains sur une image et ajoute le nom de l'action en haut à gauche de l'image.

    :param image_dir: Dossier contenant les images
    :param annotation: Annotation d'une image spécifique (élément de annolist)
    :param release_data: Structure RELEASE contenant les actions
    :param output_dir: Dossier de sauvegarde des images annotées
    """
    # Vérifier que l'annotation contient une image
    if 'image' not in annotation.dtype.names:
        print("Aucune image associée à cette annotation.")
        return

    # Extraire le nom de l'image
    image_name = annotation['image'][0, 0]['name'][0]
    
    # Charger l'image correspondante
    image_path = os.path.join(image_dir, image_name)
    image = cv2.imread(image_path)
    
    if image is None:
        print(f"Image {image_name} non trouvée.")
        return

    # Récupérer le champ act de RELEASE
    img_idx = annotation['image'][0, 0]['name'][0]
    act_info = release_data['act'][img_idx]  # Récupérer les informations de l'action

    if act_info.size > 0:
        act_name = act_info[0]['act_name'][0]  # Nom de l'action
        cat_name = act_info[0]['cat_name'][0]  # Nom de la catégorie

        # Créer un dossier pour la catégorie si nécessaire
        category_folder = os.path.join(output_dir, cat_name)
        if not os.path.exists(category_folder):
            os.makedirs(category_folder)

        # Ajouter le nom de l'action en haut à gauche de l'image
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(image, act_name, (10, 30), font, 1, (255, 255, 255), 2, cv2.LINE_AA)

    # Vérifier si l'annotation contient des personnes annotées
    if 'annorect' in annotation.dtype.names and annotation['annorect'].size > 0:
        annorect = annotation['annorect'][0]  # Liste des personnes annotées

        for person in annorect:
            if 'annopoints' in person.dtype.names and person['annopoints'].size > 0:
                annopoints = person['annopoints'][0, 0]  # Extraire la structure annopoints
                if 'point' in annopoints.dtype.names and annopoints['point'].size > 0:
                    keypoints = annopoints['point'][0]  # Liste des points clés
                    
                    # Stocker les points clés dans un dictionnaire
                    points_dict = {}

                    for point in keypoints:
                        joint_id = int(point['id'][0, 0])  # ID de l'articulation
                        x, y = int(point['x'][0, 0]), int(point['y'][0, 0])
                        points_dict[joint_id] = (x, y)

                        # Dessiner chaque articulation
                        cv2.circle(image, (x, y), 5, (0, 255, 0), -1)  # Cercle vert

                    # Dessiner les lignes reliant les articulations
                    for joint1, joint2 in skeleton_connections:
                        if joint1 in points_dict and joint2 in points_dict:
                            cv2.line(image, points_dict[joint1], points_dict[joint2], (255, 0, 0), 2)  # Ligne bleue

    # Sauvegarder l'image annotée dans le dossier correspondant à la catégorie
    output_path = os.path.join(category_folder, image_name)
    cv2.imwrite(output_path, image)
    print(f"Image annotée sauvegardée dans {category_folder}: {image_name}")
